# Falling Tablecloth

From our understanding of the Ising model, where we sampled the complex space of spins using the Metropolis-Hastings algorithm, we branch in two directions: 1) estimating probability distribution functions, and 2) simulated annealing. This example is drawn from the second, an optimization of a high-dimensional function using methods that are similar to the physics of slowly cooling hot matter. In spite of it seeming a stretch, it is closer to our Ising model than is the first, which requires generalization of the Boltzmann function and understanding of Bayes Theorem.

At a high level, the idea of simulated annealing is to take system that has many degrees of freedom, perturb one at random, and see how the energy changes as a result. The high degrees of freedom is important, there are many superior ways of optimizing functions of just a few variables. If the random change produces a lower energy ($\Delta E < 0$), accept the change, if the energy increases accept it with a probability proportional to the Boltzmann distribution; accept $\propto e^{-\frac{\Delta E}{kT}}$. Sound familiar? It should. When we apply this to a function, it's called ''simulated annealing'' because we find the minimum from 'cooling' the system. This suggests that now, in addition to accepting or rejecting proposals with the Boltzmann distribution, we also decrease $kT$, trying to allow the system to find its way to a lowest energy configuration.

To grasp the concept, maybe it will be easier to do this yourself. The following problem asks you to find the shape of a cloth that is supported at its corners, and is under the influence of gravity. The result has a nice visual interpretation that will aid your inquiry because you'll know what looks right.

#### The Change in Energy
As with any problem of this type, we begin by specifying an energy. Here, we imagine the tablecloth consists of lattice of masses connected by springs. All masses and spring are the same. The springs have an equilibrium length (length where springs exert no force) of $l$ and a spring constant $k$. If we move a particular particle a random amount $\Delta \mathbf{r} = (\Delta x, \Delta y, \Delta z)$ we get a new position $\mathbf{r}' = \mathbf{r} + \Delta\mathbf{r}$. The change in energy is computed from changes in spring lengths caused by moving the particle with respect to its four nearest neighbors, as well as changes in gravitational potential,
$$ \Delta E = -\frac{1}{2}k \sum_{i = nn} [(d'_i -l)^2 - (d_i-l)^2] + mg\Delta z.$$
Where $d'_i = |\mathbf{r}' - \mathbf{r}_i|$, the distance of the perturbed position and nearest neighbor $i$, and $d_i = |\mathbf{r} - \mathbf{r}_i|$ is the distance between the unperturbed particle and its neighbor $i$. The gravitational potential energy, or $mg\Delta z$ is included outside the sum as an additional term.

#### Problem Statement
Simulate the hanging tablecloth problem, and find the minimum energy by thermal relaxation. Assume that each of the four corners of the cloth are fixed at the same $z$ for the entire simulation and that the cloth is square.

a) Design a Metropolis updater using the formula above to find $\Delta E$. Monitor the energy as a function of MCMC steps. Assume the initial energy is zero, and keep track of total energy by summing the accepted $\Delta E$s. The shape of this curve should be evaluated to find when the function is minimized.

Convergence can be improved by selecting only a single direction to perturb in, and sampling more frequently in the $z$ direction, where potential energy can have a large impact on $\Delta E$. So, 25% of the time perturb in the $x$ direction, 25% in the $y$, and 50% in the $z$. The perturbations are going to be in space, and some normal distribution is the right choice. Let's increase the perturbation distribution by a factor of two in the $z$ direction so it is explored more aggressively. Something like the following in code:

In [6]:
import numpy as np
sigma = .25 # A choice of standard deviation for the random perturbations.
dx,dy,dz = 0,0,0
p = np.random.rand()
if p<.25:
    dx = np.random.randn() * sigma
elif p<.5:
    dy = np.random.randn() * sigma
else:
    dz = np.random.randn() * 2 * sigma

So that we all agree on the results, let's have the following set of values for $N$, $k$, $l$, $m$, $\sigma$, $g$, and $kT$.

In [1]:
N = 16 # Number of masses per side.
k=50   # Spring constant
l=1    # Spring equilibrium length
m=.04  # Mass, almost looks like a density in the limit of N -> inf
sigma = .25*l  # This is standard deviation of the gaussian region sampled for perturbations.
g = 9.8        # graviational acceleration, note sign!
kT = 1e-3      # Temparature appearing in Boltzmann factor, this should be changed through runs.

Run the simulation to equilibrium and plot the result. Try the `plot_wireframe` function. Begin with all positions of masses seperated by $l$ from nearest neighbors, and having the same $z$ coordinate. 

How did you determine the system was in equilibrium? Add this function to your code.

Repeat the experiment, but now include a ''cooling schedule''. Try halving the temperature each time the system reaches equilibrium. Do three successive halvings. Plot the different final configurations and comment on their differences.

(b) Plot the energy per particle vs the temperature at various lattice sizes. Explain what you find.

In [ ]:
class Cloth():
    def __init__():
        
    def do_mcmc_step(self):
        for i in range(self.L**2):
            dE,pt,dr = self.perturb()
            if dE<0 or np.random.rand() < np.exp(-dE/self.kT):
                self.cloth[pt[0],pt[1],:] += dr
                self.E += dE
            self.step += 1
            
    def calc_dE(self,pt,dr):
        # prep lists, write new function to get nn
        # nn = nearest neighbors - can be 3 or 4 in list
        # prep vectors by pulling them out of self.cloth
        # see sum in delta E
        for n in nn:
            term1 = (np.linalg.norm(rp-nr)-self.l)**2
            # rp = r prime the perturbed postition
            # nr = neighbor r
            term2 = # unperturbed distance to 
        # multiply by -.5*k
        # add mg dz